# Going deeper into Parser

## The main uc.py module

First, in order to standardize the way the compiler we are building is called, especially when submitting your project to Susy, I am suggesting that you use the module below:

In [ ]:
#!/usr/bin/env python3
# ============================================================
# uc.py -- uC (a.k.a. micro C) language compiler
#
# This is the main program for the uc compiler, which just
# parses command-line options, figures out which source files
# to read and write to, and invokes the different stages of
# the compiler proper.
# ============================================================

import sys
from contextlib import contextmanager
from uc_parser import UCParser

"""
One of the most important (and difficult) parts of writing a compiler
is reliable reporting of error messages back to the user.  This file
defines some generic functionality for dealing with errors throughout
the compiler project. Error handling is based on a subscription/logging
based approach.

To report errors in uc compiler, we use the error() function. For example:

       error(lineno,"Some kind of compiler error message")

where lineno is the line number on which the error occurred.

Error handling is based on a subscription based model using context-managers
and the subscribe_errors() function. For example, to route error messages to
standard output, use this:

       with subscribe_errors(print):
            run_compiler()

To send messages to standard error, you can do this:

       import sys
       from functools import partial
       with subscribe_errors(partial(print,file=sys.stderr)):
            run_compiler()

To route messages to a logger, you can do this:

       import logging
       log = logging.getLogger("somelogger")
       with subscribe_errors(log.error):
            run_compiler()

To collect error messages for the purpose of unit testing, do this:

       errs = []
       with subscribe_errors(errs.append):
            run_compiler()
       # Check errs for specific errors

The utility function errors_reported() returns the total number of
errors reported so far.  Different stages of the compiler might use
this to decide whether or not to keep processing or not.

Use clear_errors() to clear the total number of errors.
"""

_subscribers = []
_num_errors = 0


def error(lineno, message, filename=None):
    """ Report a compiler error to all subscribers """
    global _num_errors
    if not filename:
        errmsg = "{}: {}".format(lineno, message)
    else:
        errmsg = "{}:{}: {}".format(filename,lineno,message)
    for subscriber in _subscribers:
        subscriber(errmsg)
    _num_errors += 1


def errors_reported():
    """ Return number of errors reported. """
    return _num_errors


def clear_errors():
    """ Clear the total number of errors reported. """
    global _num_errors
    _num_errors = 0


@contextmanager
def subscribe_errors(handler):
    """ Context manager that allows monitoring of compiler error messages.
        Use as follows where handler is a callable taking a single argument
        which is the error message string:

        with subscribe_errors(handler):
            ... do compiler ops ...
    """
    _subscribers.append(handler)
    try:
        yield
    finally:
        _subscribers.remove(handler)


class Compiler:
    """ This object encapsulates the compiler and serves as a
        facade interface for the compiler itself.
    """

    def __init__(self):
        self.total_errors = 0
        self.total_warnings = 0

    def _parse(self, susy, ast_file, debug):
        """ Parses the source code. If ast_file != None,
            or running at susy machine,
            prints out the abstract syntax tree.
        """
        self.parser = UCParser()
        self.ast = self.parser.parse(self.code, '', debug)
        if susy:
            self.ast.show(showcoord=True)
        elif ast_file is not None:
            self.ast.show(buf=ast_file, showcoord=True)

    def _do_compile(self, susy, ast_file, debug):
        """ Compiles the code to the given file object. """
        self._parse(susy, ast_file, debug)

    def compile(self, code, susy, ast_file, debug):
        """ Compiles the given code string """
        self.code = code
        with subscribe_errors(lambda msg: sys.stderr.write(msg+"\n")):
            self._do_compile(susy, ast_file, debug)
            if errors_reported():
                sys.stderr.write("{} error(s) encountered.".format(errors_reported()))
        return 0


def run_compiler():
    """ Runs the command-line compiler. """

    if len(sys.argv) < 2:
        print("Usage: ./uc.py <source-file> [-at-susy] [-no-ast] [-debug]")
        sys.exit(1)

    emit_ast = True
    susy = False
    debug = False

    params = sys.argv[1:]
    files = sys.argv[1:]

    for param in params:
        if param[0] == '-':
            if param == '-no-ast':
                emit_ast = False
            elif param == '-at-susy':
                susy = True
            elif param == '-debug':
                debug = True
            else:
                print("Unknown option: %s" % param)
                sys.exit(1)
            files.remove(param)

    for file in files:
        if file[-3:] == '.uc':
            source_filename = file
        else:
            source_filename = file + '.uc'

        open_files = []
        ast_file = None
        if emit_ast and not susy:
            ast_filename = source_filename[:-3] + '.ast'
            print("Outputting the AST to %s." % ast_filename)
            ast_file = open(ast_filename, 'w')
            open_files.append(ast_file)

        source = open(source_filename, 'r')
        code = source.read()
        source.close()

        retval = Compiler().compile(code, susy, ast_file, debug)
        for f in open_files:
            f.close()
        if retval != 0:
            sys.exit(retval)

    sys.exit(retval)


if __name__ == '__main__':
    run_compiler()

## Building AST nodes

### Building Declarations

At uC, statements can come several in a line. For example:
```
  int x, y, z = 5;
```
However, for AST, we must divide them to separate the Decl nodes.
The code snippet below splits the declarations and always returns a list of Decl nodes, even if it is a single element.

In [ ]:
    def _build_declarations(self, spec, decls):
        """ Builds a list of declarations all sharing the given specifiers.
        """
        declarations = []

        for decl in decls:
            assert decl['decl'] is not None
            declaration = uc_ast.Decl(
                    name=None,
                    type=decl['decl'],
                    init=decl.get('init'),
                    coord=decl['decl'].coord)

            fixed_decl = self._fix_decl_name_type(declaration, spec)
            declarations.append(fixed_decl)

        return declarations

Due to the order in which declarators are constructed, they have to be fixed in order to look like a normal AST.

When a declaration arrives from syntax construction, it has these problems:
 * The innermost VarDecl has no type (because the basic type is only known at the uppermost declaration level)
 * The declaration has no variable name since that is saved in the innermost VarDecl

The method below fixes these problems:

In [ ]:
    def _fix_decl_name_type(self, decl, typename):
        """ Fixes a declaration. Modifies decl.
        """
        # Reach the underlying basic type
        type = decl
        while not isinstance(type, uc_ast.VarDecl):
            type = type.type

        decl.name = type.declname

        # The typename is a list of types. If any type in this
        # list isn't an Type, it must be the only
        # type in the list.
        # If all the types are basic, they're collected in the
        # Type holder.
        for tn in typename:
            if not isinstance(tn, uc_ast.Type):
                if len(typename) > 1:
                    self._parse_error(
                        "Invalid multiple types specified", tn.coord)
                else:
                    type.type = tn
                    return decl

        if not typename:
            # Functions default to returning int
            if not isinstance(decl.type, uc_ast.FuncDecl):
                self._parse_error("Missing type in declaration", decl.coord)
            type.type = uc_ast.Type(['int'], coord=decl.coord)
        else:
            # At this point, we know that typename is a list of Type
            # nodes. Concatenate all the names into a single list.
            type.type = uc_ast.Type(
                [typename.names[0]],
                coord=typename.coord)
        return decl

The AST for the statement: ```int x, y, z = 5;```look like this:
````
Program: 
    GlobalDecl: 
        Decl: ID(name='x'  )
            VarDecl: 
                Type: ['int']   @ 1:1
        Decl: ID(name='y'  )
            VarDecl: 
                Type: ['int']   @ 1:1
        Decl: ID(name='z'  )
            VarDecl:
                Type: ['int']   @ 1:1
            Constant: int, 5   @ 1:15
```

### Type Modifier

A uC type consists of a basic type declaration, with a list of modifiers. For example:
```  
int c[5];
```
The basic declaration here is 'int c', and the array is the modifier.
Basic declarations are represented by VarDecl and the modifiers are **FuncDecl**, **PtrDecl** and **ArrayDecl**.

The standard states that whenever a new modifier is parsed, it should be added to the end of the list of modifiers. For example:

**Array Declarators**

In a declaration T D where D has the form D1 [constant_expression_opt] and the type of the identifier in the declaration T D1 is "type-modifier T", the type of the identifier of D is "type-modifier array of T"

This is what the method below does. The declarator it receives can be a list of declarators ending with VarDecl. It tacks the modifier to the end of this list, just before the VarDecl.

In [ ]:
    def _type_modify_decl(self, decl, modifier):
        """ Tacks a type modifier on a declarator, and returns
            the modified declarator.
            Note: the declarator and modifier may be modified
        """
        modifier_head = modifier
        modifier_tail = modifier

        # The modifier may be a nested list. Reach its tail.
        while modifier_tail.type:
            modifier_tail = modifier_tail.type

        # If the decl is a basic type, just tack the modifier onto it
        if isinstance(decl, uc_ast.VarDecl):
            modifier_tail.type = decl
            return modifier
        else:
            # Otherwise, the decl is a list of modifiers. Reach
            # its tail and splice the modifier onto the tail,
            # pointing to the underlying basic type.
            decl_tail = decl

            while not isinstance(decl_tail.type, uc_ast.VarDecl):
                decl_tail = decl_tail.type

            modifier_tail.type = decl_tail.type
            decl_tail.type = modifier_head
            return decl

The AST for the statement ```int c[5];``` look like this:

```
Program: 
    GlobalDecl: 
        Decl: ID(name='c'  )
            ArrayDecl: 
                VarDecl:
                    Type: ['int']   @ 1:1
                Constant: int, 5   @ 1:7
```